# 07 - Generate statistics CSVs
Creating a separate notebook to calculate the statistics all in one pass so that I can complete a representative set of CSVs for each instrument class.

### Statistics for QARTOD tests in production

In [1]:
# Import libraries available from main conda channels or conda-forge
import xarray as xr
import pandas as pd
import numpy as np
import glob
import os
import re
import warnings
warnings.filterwarnings("ignore")

# Import dask tools and ProgressBar
import dask
from dask.diagnostics import ProgressBar

# Import qartod_testing project functions
from qartod_testing.data_processing import ooinet_gold_copy_request, get_test_parameters, parse_qartod_executed, qartod_summary_expanded

# Import OOI library functions
from ooi_data_explorations.common import merge_frames
from ooinet.M2M import get_deployments, get_annotations

In [2]:
# Setup parameters needed to request data
refdes = "GS01SUMO-SBD12-04-PCO2AA000"
method = "recovered_host"
stream = "pco2a_a_dcl_instrument_air_recovered"

# Site, node, and sensor info from deconstructed reference designator
# [site, node, sensor] = refdes.split('-', 2)

In [3]:
# Routine in data_processing module from this project to download the gold copy THREDDs datasets
# Variable 'files' contains list of catalog URLs for downloaded datasets 
files = ooinet_gold_copy_request(refdes, method, stream)

In [4]:
# Load expected results data from external data folder
folder_path = os.path.join(os.path.abspath('../data/external'), method, stream, refdes)
expected_files = glob.glob(folder_path+'/*.nc')
expected_files.sort() # sorts local test files in alphanumeric order

In [5]:
expected_files
# get_deployments(refdes)

['/home/jovyan/code/qartod_testing/data/external/recovered_host/pco2a_a_dcl_instrument_air_recovered/GS01SUMO-SBD12-04-PCO2AA000/deployment0001_GS01SUMO-SBD12-04-PCO2AA000-recovered_host-pco2a_a_dcl_instrument_air_recovered_20150225T032326.197000-20150322T233222.814000.nc',
 '/home/jovyan/code/qartod_testing/data/external/recovered_host/pco2a_a_dcl_instrument_air_recovered/GS01SUMO-SBD12-04-PCO2AA000/deployment0001_GS01SUMO-SBD12-04-PCO2AA000-recovered_host-pco2a_a_dcl_instrument_air_recovered_20150323T003223.329000-20150419T234204.459000.nc',
 '/home/jovyan/code/qartod_testing/data/external/recovered_host/pco2a_a_dcl_instrument_air_recovered/GS01SUMO-SBD12-04-PCO2AA000/deployment0001_GS01SUMO-SBD12-04-PCO2AA000-recovered_host-pco2a_a_dcl_instrument_air_recovered_20150420T004204.978000-20150517T235146.471000.nc',
 '/home/jovyan/code/qartod_testing/data/external/recovered_host/pco2a_a_dcl_instrument_air_recovered/GS01SUMO-SBD12-04-PCO2AA000/deployment0001_GS01SUMO-SBD12-04-PCO2AA000-rec

In [6]:
get_annotations(refdes)

,@class,id,subsite,node,sensor,method,stream,beginDT,endDT,annotation,exclusionFlag,source,qcFlag,parameters
0,.AnnotationRecord,95,GS01SUMO,SBD12,None,None,None,1450051200000,1481500800000,"Upon recovery of deployment 2, damage to some ...",False,lgarzio@marine.rutgers.edu,0,[]
1,.AnnotationRecord,576,GS01SUMO,SBD12,04-PCO2AA000,None,None,1480036260000,1544330520000,Deployment 3: UPDATED 2020-05-11: Upon recover...,False,lgarzio@marine.rutgers.edu,9,[]
2,.AnnotationRecord,834,GS01SUMO,None,None,None,None,1512777600000,1544330520000,Deployment 3: *UPDATED 2020-04-28: This moorin...,False,lgarzio@marine.rutgers.edu,0,[]
3,.AnnotationRecord,1579,GS01SUMO,SBD12,04-PCO2AA000,None,None,1553212800000,1579514460000,Deployment 4: Data range suspect beginning 3/2...,False,cdobson@whoi.edu,0,[]


#### Gross range test statistics

In [7]:
def collect_statistics(file_paths, test_name):
    """
    Calls other functions to calculate statistics from a set of files and a name of a QARTOD test. The statistics are organized in a DataFrame.
    
    Parameters:
    -----------
        file_paths: list of paths to each file that will have statistics calculated. File names must include "deployment00##".
        test_name: string of QARTOD test name, i.e. "gross_range", "climatology".
        
    Returns:
    --------
        statistics: Pandas DataFrame containing statistics on each parameter with a QARTOD test in order of deployment number, then statistics of the full record.
        
    Version 23 Aug 2023, Kylene M Cooley    
    """
    
    # Initialize empty dictionary for statistics
    statistics = {}
    
    # Create a copy of list of file paths for individual deployment statistics
    paths_copy = file_paths.copy()
    m = 0

    while len(paths_copy)>0:
        file = paths_copy[0]

        # get deployment from current file, then open local test and expected test datasets
        deployment = re.findall('deployment00[0-2][0-9]', file)[0][-2:]
        # here figure out how to get all files of a single deployment and do a merged dataset like below
        files_with_deployment_num = [x for x in paths_copy if f'deployment00{deployment}' in x]
        
        if len(files_with_deployment_num)>1:
            file_ds = [xr.open_dataset(single_file) for single_file in files_with_deployment_num]
            file_ds = merge_frames(file_ds)
        else:  
            file_ds = xr.open_dataset(file)

        # Get parameters that have QARTOD executed from expected test dataset
        test_parameters = get_test_parameters(file_ds)
        parameters = list(test_parameters.keys())

        # Separate QARTOD test flags in expected test dataset by QARTOD test name
        file_ds = parse_qartod_executed(file_ds, parameters)

        # Update summary statistics dictionary for each deployment, then for all deployments
        print("Evaluating statistics on QARTOD flags for deployment "f"{deployment}")
        summary_results = qartod_summary_expanded(file_ds, parameters, deployment, test_name)
        statistics.update({f"{m}" : summary_results })
        
        for x in files_with_deployment_num:
            paths_copy.remove(x)
        m += 1

    # Add entry to summary statistics for full data record after last deployments
    # Open all expected data files and create merged full dataset
    merged_ds = [xr.open_dataset(single_file) for single_file in file_paths]
    merged_ds = merge_frames(merged_ds)
    deployment = "all"

    # Summary of flags from merged dataset
    print("Evaluating statistics on QARTOD flags for all deployments")
    merged_ds = parse_qartod_executed(merged_ds, parameters)
    summary_results = qartod_summary_expanded(merged_ds, parameters, deployment, test_name)
    statistics.update({ "all" : summary_results })

    # Create data frame from dictionary and check contents
    statistics = pd.DataFrame.from_dict(statistics, orient='index')
    statistics = statistics.set_index('deployment')
    return statistics

In [8]:
gross_range_stats = collect_statistics(expected_files, "gross_range")
gross_range_stats

Evaluating statistics on QARTOD flags for deployment 01
Evaluating statistics on QARTOD flags for deployment 02
Evaluating statistics on QARTOD flags for deployment 04
Evaluating statistics on QARTOD flags for all deployments


,partial_pressure_co2_atm total,atm good,atm good %,atm suspect,atm suspect %,atm fail,atm fail %
deployment,,,,,,,
01,17847,11529,64.60,6318,35.4,0,0.0
02,87065,61640,70.80,25425,29.2,0,0.0
04,27342,27306,99.87,0,0.0,0,0.0
all,132254,100475,75.97,31743,24.0,0,0.0


In [9]:
# Save data frames as CSVs
folder_path = os.path.join(os.path.abspath('../data/processed'), method, stream, refdes)
os.makedirs(folder_path, exist_ok=True)
gross_range_stats.to_csv(folder_path+f"/gross_range-{refdes}-flag_statistics.csv", na_rep='NaN', mode='w')

#### Climatology test statistics

In [10]:
climatology_stats = collect_statistics(expected_files, "climatology")
climatology_stats

Evaluating statistics on QARTOD flags for deployment 01
Evaluating statistics on QARTOD flags for deployment 02
Evaluating statistics on QARTOD flags for deployment 04
Evaluating statistics on QARTOD flags for all deployments


,partial_pressure_co2_atm total,atm good,atm good %,atm suspect,atm suspect %,atm fail,atm fail %
deployment,,,,,,,
01,17847,11527,64.59,6320,35.41,0,0.0
02,87065,53435,61.37,33630,38.63,0,0.0
04,27342,27306,99.87,36,0.13,0,0.0
all,132254,92268,69.77,39986,30.23,0,0.0


In [11]:
# Save data frames as CSVs
folder_path = os.path.join(os.path.abspath('../data/processed'), method, stream, refdes)
os.makedirs(folder_path, exist_ok=True)
climatology_stats.to_csv(folder_path+f"/climatology-{refdes}-flag_statistics.csv", na_rep='NaN', mode='w')